# Testing logic to clean statistics data

In [34]:
import numpy as np
import pandas as pd
import re
import pcap_processor

In [35]:
data = []
with open("system_input/udp_stats.txt") as file:
    data = list(file)

In [36]:
print(data[:5])

['================================================================================\n', 'UDP Conversations\n', 'Filter:<No Filter>\n', '                                                           |       <-      | |       ->      | |     Total     |    Relative    |   Duration   |\n', '                                                           | Frames  Bytes | | Frames  Bytes | | Frames  Bytes |      Start     |              |\n']


In [37]:
data = data[5:]

In [38]:
print(data[9])

fe80::c18:8b73:1e6e:f116:5353 <-> ff02::fb:5353                    0         0     103     41670     103     41670     8.839083000      2969.2724



In [8]:
data = [e.split(" ") for e in data]

In [9]:
print(data)

[['192.168.1.146:64146', '', '', '', '', '', '', '', '<->', '74.125.197.105:443', '', '', '', '', '', '', '', '', '', '', '', '', '341', '', '', '', '380549', '', '', '', '', '118', '', '', '', '', '21012', '', '', '', '', '459', '', '', '', '401561', '', '', '662.172540000', '', '', '', '', '', '', '', '18.6986\n'], ['192.168.1.145:49154', '', '', '', '', '', '', '', '<->', '255.255.255.255:6666', '', '', '', '', '', '', '', '', '', '', '', '', '0', '', '', '', '', '', '', '', '', '0', '', '', '', '', '408', '', '', '', '', '88536', '', '', '', '', '408', '', '', '', '', '88536', '', '', '', '', '1.835876000', '', '', '', '', '', '3033.0915\n'], ['192.168.1.198:49154', '', '', '', '', '', '', '', '<->', '255.255.255.255:6666', '', '', '', '', '', '', '', '', '', '', '', '', '0', '', '', '', '', '', '', '', '', '0', '', '', '', '', '406', '', '', '', '', '88102', '', '', '', '', '406', '', '', '', '', '88102', '', '', '', '', '1.078166000', '', '', '', '', '', '3033.0286\n'], ['192.168

In [10]:
test_element = data[0]

In [11]:
test_element = [e for e in test_element if e != '']

In [12]:
test_element = [e for e in test_element if e != '<->'] 

In [13]:
'' in test_element

False

In [14]:
print(test_element)

['192.168.1.146:64146', '74.125.197.105:443', '341', '380549', '118', '21012', '459', '401561', '662.172540000', '18.6986\n']


In [15]:
test_element = [e.split(":") for e in test_element]

In [16]:
print(test_element)

[['192.168.1.146', '64146'], ['74.125.197.105', '443'], ['341'], ['380549'], ['118'], ['21012'], ['459'], ['401561'], ['662.172540000'], ['18.6986\n']]


In [17]:
_ = []
for e in test_element:
    _ += e
test_element = _

In [18]:
print(test_element)

['192.168.1.146', '64146', '74.125.197.105', '443', '341', '380549', '118', '21012', '459', '401561', '662.172540000', '18.6986\n']


In [19]:
for i,e in enumerate(test_element):
    if re.search("\\n", e) is not None:
        test_element[i] = e.split("\n")[0]

In [20]:
print(len(test_element))

12


In [25]:
labels = ["src_ip", "src_port", "dst_ip, dst_port", "src_frames", "src_bytes", "dst_frames", "dst_bytes", "total_frames", "total_bytes", "relative_start", "duration"]
print(len(test_element))

12


In [65]:
omit = ['', '|', '->', '<-', '|\n']
labels = [i for i in labels if i not in omit]

# Working with the new pcap_processor class

In [47]:
import numpy as np
import pandas as pd
import re
import pcap_processor
import copy

In [31]:
labels = ["src_ip", "src_port", "dst_ip", "dst_port", "src_frames", "src_bytes", "dst_frames", "dst_bytes", "total_frames", "total_bytes", "relative_start", "duration"]

In [48]:
processor_tcp = pcap_processor.processorV2("system_input/tcp_stats.txt")
processor_udp = pcap_processor.processorV2("system_input/udp_stats.txt")


[+] Reading File

[+] File has been read into memory

[+] Reading File

[+] File has been read into memory


In [49]:
clean_tcp_stats = processor_tcp.clean()


[+] Opening file for cleaning

[+] Locating unstructured column labels
['================================================================================\n', 'TCP Conversations\n', 'Filter:<No Filter>\n', '                                                           |       <-      | |       ->      | |     Total     |    Relative    |   Duration   |\n', '                                                           | Frames  Bytes | | Frames  Bytes | | Frames  Bytes |      Start     |              |\n']

[+] Removing unstructured column labels

[+] Current state:
192.168.1.146:59666        <-> 35.155.143.197:443              80      8404      83      7087     163     15491    29.991524000      2927.6352


[+] Cleaning dataset step 1/2
['192.168.1.146:59666', '', '', '', '', '', '', '', '<->', '35.155.143.197:443', '', '', '', '', '', '', '', '', '', '', '', '', '', '80', '', '', '', '', '', '8404', '', '', '', '', '', '83', '', '', '', '', '', '7087', '', '', '', '', '163', '', '', '', ''

In [50]:
clean_udp_stats = processor_udp.clean()


[+] Opening file for cleaning

[+] Locating unstructured column labels
['================================================================================\n', 'UDP Conversations\n', 'Filter:<No Filter>\n', '                                                           |       <-      | |       ->      | |     Total     |    Relative    |   Duration   |\n', '                                                           | Frames  Bytes | | Frames  Bytes | | Frames  Bytes |      Start     |              |\n']

[+] Removing unstructured column labels

[+] Current state:
fe80::c18:8b73:1e6e:f116:5353 <-> ff02::fb:5353                    0         0     103     41670     103     41670     8.839083000      2969.2724


[+] Cleaning dataset step 1/2
['fe80::c18:8b73:1e6e:f116:5353', '<->', 'ff02::fb:5353', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '0', '', '', '', '', '', '', '', '', '0', '', '', '', '', '103', '', '', '', '', '41670', '', '', '', '', '103', '', '', 

In [51]:
print(clean_tcp_stats[:2])

          src_ip src_port          dst_ip dst_port  src_frames  src_bytes  \
0  192.168.1.146    59787  192.30.252.154       80         615     910519   
1  192.168.1.146    59635  23.222.154.141      443         133     143779   

   dst_frames  dst_bytes  total_frames  total_bytes  relative_start  duration  
0         395      27638          1010       938157      748.442405   57.9930  
1         151      25693           284       169472        0.469667    5.1496  


In [52]:
print(clean_udp_stats)

                       src_ip src_port           dst_ip dst_port  src_frames  \
0               192.168.1.146    64146   74.125.197.105      443         341   
1               192.168.1.145    49154  255.255.255.255     6666           0   
2               192.168.1.198    49154  255.255.255.255     6666           0   
3               192.168.1.146    63247   74.125.197.105      443         235   
4               192.168.1.192    49347  239.255.255.250     1900           0   
5               192.168.1.137    17500  255.255.255.255    17500           0   
6               192.168.1.146    50920    172.217.4.206      443         120   
7               192.168.1.146    61788  108.177.112.189      443          62   
8               192.168.1.146    60111   74.125.124.189      443          64   
9    fe80::c18:8b73:1e6e:f116     5353         ff02::fb     5353           0   
10              192.168.1.146    49449   209.85.145.189      443          54   
11              192.168.1.146     5353  

In [35]:
clean_tcp_stats.describe()

,src_frames,src_bytes,dst_frames,dst_bytes,total_frames,total_bytes,relative_start,duration
count,293.000000,293.000000,293.000000,293.000000,293.000000,293.000000,293.000000,293.000000
mean,19.095563,10179.344710,21.208191,4006.750853,40.303754,14186.095563,1207.415411,230.837514
std,40.173226,55937.597166,30.748555,7396.454078,70.066797,58192.854937,1042.354789,474.123838
min,0.000000,0.000000,1.000000,54.000000,1.000000,54.000000,0.000000,0.000000
25%,4.000000,271.000000,8.000000,660.000000,10.000000,780.000000,214.724186,0.488200
50%,12.000000,4505.000000,15.000000,2007.000000,26.000000,7513.000000,1143.740392,44.818400
75%,22.000000,6923.000000,25.000000,5046.000000,47.000000,11966.000000,1705.959666,220.855300
max,615.000000,910519.000000,395.000000,92034.000000,1010.000000,938157.000000,3032.452213,2984.217700


In [12]:
clean_udp_stats.describe()

,src_frames,src_bytes,dst_frames,dst_bytes,total_frames,total_bytes,relative_start,duration
count,661.000000,661.000000,661.000000,661.000000,661.000000,661.000000,661.000000,661.000000
mean,4.166415,2258.102874,7.425113,2000.944024,11.591528,4259.046899,1125.790363,136.248824
std,17.991652,17906.142386,28.683739,6997.223877,37.670495,20653.053461,947.945741,563.732576
min,0.000000,0.000000,0.000000,0.000000,1.000000,67.000000,0.209073,0.000000
25%,0.000000,0.000000,1.000000,79.000000,2.000000,187.000000,296.550867,0.017000
50%,1.000000,119.000000,1.000000,90.000000,2.000000,297.000000,823.132458,0.115100
75%,1.000000,289.000000,4.000000,1011.000000,4.000000,1828.000000,1703.933854,2.966600
max,341.000000,380549.000000,408.000000,88536.000000,459.000000,401561.000000,3035.726296,3033.091500


In [36]:
print(clean_tcp_stats["src_ip"])

0      192.168.1.146
1      192.168.1.146
2      192.168.1.146
3      192.168.1.146
4      192.168.1.146
5      192.168.1.146
6      192.168.1.146
7      192.168.1.146
8      192.168.1.146
9      192.168.1.146
10     192.168.1.146
11     192.168.1.146
12     192.168.1.146
13     192.168.1.146
14     192.168.1.146
15     192.168.1.146
16     192.168.1.146
17     192.168.1.146
18     192.168.1.146
19     192.168.1.146
20     192.168.1.146
21     192.168.1.146
22     192.168.1.146
23     192.168.1.146
24     192.168.1.146
25     192.168.1.146
26     192.168.1.146
27     192.168.1.146
28     192.168.1.146
29     192.168.1.146
           ...      
263    192.168.1.146
264    192.168.1.146
265    192.168.1.146
266    192.168.1.146
267    192.168.1.146
268    192.168.1.146
269    192.168.1.146
270    192.168.1.146
271    192.168.1.146
272    192.168.1.146
273    192.168.1.146
274    192.168.1.146
275    192.168.1.146
276    192.168.1.146
277    192.168.1.146
278    192.168.1.146
279    192.16

In [61]:
clean_tcp_stats.dst_ip.unique()

array(['192.30.252.154', '23.222.154.141', '52.109.76.32',
       '18.233.104.54', '35.155.143.197', '172.217.4.110',
       '52.27.211.200', '52.32.16.231', '52.89.46.250', '18.236.57.250',
       '216.58.192.206', '54.81.141.213', '18.204.41.111', '13.32.167.89',
       '108.177.112.108', '173.194.198.109', '52.84.19.121',
       '17.249.44.80', '54.197.54.176', '52.54.211.148', '151.101.129.69',
       '35.173.144.198', '17.249.44.85', '172.217.4.69', '17.249.44.101',
       '23.20.235.164', '172.217.9.46', '173.194.192.108',
       '34.236.76.192', '172.217.8.206', '107.23.246.53', '172.217.4.78',
       '162.125.34.129', '17.249.44.24', '54.209.146.99', '54.174.0.98',
       '34.239.4.144', '34.232.133.243', '17.248.130.143', '172.217.0.14',
       '17.249.44.77', '172.217.4.206', '107.23.203.104', '54.84.181.13',
       '17.167.192.128', '108.177.121.188', '108.177.112.109',
       '74.125.126.109', '173.194.198.108', '192.168.1.1', '13.33.144.8',
       '17.134.126.34', '17.167.

In [74]:
ip_address = []
ip_address = clean_tcp_stats.src_ip.unique().tolist()
ip_address += clean_tcp_stats.dst_ip.unique().tolist()
ip_address += clean_udp_stats.dst_ip.unique().tolist()
ip_address += clean_udp_stats.src_ip.unique().tolist()
print(len(ip_address))
look_up_table = pcap_processor.get_lookup_table(ip_address)
print(len(look_up_table))
print(look_up_table)

203
173
{'108.177.121.188': 0, '172.217.4.69': 1, '52.27.211.200': 2, 'fe80::ae89:95ff:fe4d:53b1': 3, '34.192.34.151': 4, '74.125.69.188': 5, '216.58.192.163': 6, '192.168.1.146': 7, '108.177.112.108': 8, '172.217.8.161': 9, '52.109.76.32': 10, '172.217.6.98': 11, '54.71.113.104': 12, '13.33.144.8': 13, '216.58.216.99': 14, '17.167.192.128': 15, '13.32.167.89': 16, '17.167.192.94': 17, '162.125.34.129': 18, '17.134.127.223': 19, '66.58.255.52': 20, '172.217.5.14': 21, '35.173.144.198': 22, '52.32.16.231': 23, '74.125.129.108': 24, '172.217.8.206': 25, '17.253.24.125': 26, '173.194.193.188': 27, '17.134.126.34': 28, '192.168.1.108': 29, '172.217.4.74': 30, '216.58.216.110': 31, '172.217.4.46': 32, '173.194.198.109': 33, 'ff02::1:2': 34, '35.155.143.197': 35, '172.217.0.14': 36, '255.255.255.255': 37, '17.248.130.201': 38, '209.85.147.188': 39, '17.249.44.24': 40, '239.255.255.250': 41, 'fe80::a2c9:a0ff:fef8:d6c': 42, '52.109.120.21': 43, 'ff02::fb': 44, '74.125.132.188': 45, '216.58.216

# Visualize the data

In [8]:
stats.tcp_labels

['src_ip',
 'src_port',
 'dst_ip',
 'dst_port',
 'src_frames',
 'src_bytes',
 'dst_frames',
 'dst_bytes',
 'total_frames',
 'total_bytes',
 'relative_start',
 'duration']

In [16]:
from prettytable import PrettyTable
from plotly import __version__
import plotly.offline

In [38]:
def plot_bar(x_data, y_data, x_label, y_label, title=None):
    plotly.offline.iplot({"data":[plotly.graph_objs.Bar(x=x_data,
                                                            y=y_data)], 
                          "layout":plotly.graph_objs.Layout(title=title, 
                                                            xaxis=dict(title=x_label), 
                                                            yaxis=dict(title=y_label))})

In [35]:
plot_bar(clean_stats["dst_port"], clean_stats["src_bytes"], "dst port", "source bytes", "DestinationPort/SourceBytes")

In [36]:
plot_bar(clean_stats["src_port"], clean_stats["src_bytes"], "src port", "source bytes", "SourcePort/SourceBytes")

In [40]:
plot_bar(clean_stats["relative_start"], clean_stats["total_bytes"], "Time", "Total Bytes", "Bytes by Time")

In [25]:
#Create the graph
plotly.offline.iplot({"data":[plotly.graph_objs.Scatter(x=clean_stats["src_ip"], y=clean_stats["total_frames"])], "layout":plotly.graph_objs.Layout(title="Frames per IP ", xaxis=dict(title="IP"), yaxis=dict(title="Frames"))})